In [2]:
# Imports
import requests

# Define API URLs and the API key
API_URL_MATCHES = "https://api.football-data.org/v4/matches"
API_URL_STANDINGS = "https://api.football-data.org/v4/competitions/PL/standings"
API_KEY = "e3672a3115a64db2bcc8b4040090b619"

# The year that stats will be used
YEAR = 2024

# Define the headers for the API request
headers = {
    "X-Auth-Token": API_KEY
}

# Define the parameters for the API request
params = {
    "season": YEAR
}

# Send a request to get standings data
response = requests.get(API_URL_STANDINGS, headers=headers, params=params)

# If the API request fails, print an error message and exit
if response.status_code != 200:
    print("Error fetching standings:", response.status_code)
    exit()

# Function to get detailed team information for a given team
def getTeamInfo(response, teamName):
    standings = response.json()
    standingsTotal = standings['standings'][0] # Season standings
    standingsHome = standings['standings'][1] # Season standings for home games
    standingsAway = standings['standings'][2] # Season standings for away games

    # Retrieves current form and season/home/away informations
    teamSeason = next(team for team in standingsTotal["table"] if team['team']['shortName'] == teamName)
    teamHomeSeason = next(team for team in standingsHome["table"] if team['team']['shortName'] == teamName)
    teamAwaySeason = next(team for team in standingsAway["table"] if team['team']['shortName'] == teamName)

    return [
        teamSeason["form"], # current form
        teamSeason["won"], # season total wins
        teamSeason["lost"], # season total lost
        teamSeason["draw"], # season total draws
        teamHomeSeason["won"], # season total home wins
        teamHomeSeason["lost"], # season total home lost
        teamHomeSeason["draw"], # season total home draws
        teamAwaySeason["won"], # season total away wins
        teamAwaySeason["lost"], # season total away lost
        teamAwaySeason["draw"] # season total away draws
    ]

# Function to predict match outcome based on team form, stats, home game advantage
def predictMatch(team1Form, team2Form, team1Status, team1Season, team2Season):

  # Assigns weight for each category
  formWinWeight = 5
  formDrawWeight = 2
  winWeight = 5
  drawWeight = 2
  loseWeight = 2
  homeAdvantage = 0.15

  # Initialize score for both teams
  team1_total = 0
  team2_total = 0

  # Calculate score for team's current form
  team1_form_score = (team1Form.count("W") * formWinWeight) + (team1Form.count("D") * formDrawWeight)
  team2_form_score = (team2Form.count("W") * formWinWeight) + (team1Form.count("D") * formDrawWeight)
  team1_total += team1_form_score
  team2_total += team2_form_score

  # Calculate score for team's season performance

  # Team 1
  if ((team1Season[0]*winWeight) - (team1Season[1]*loseWeight)) <= 0:
    team1_season_score = 0
  else:
    team1_season_score = ((team1Season[0]*winWeight) + (team1Season[2]*drawWeight)) - (team1Season[1]*loseWeight)

  team1_total += team1_season_score

  # Team 2
  if ((team2Season[0]*winWeight) - (team2Season[1]*loseWeight)) <= 0:
    team2_season_score = 0
  else:
    team2_season_score = ((team2Season[0]*winWeight) + (team2Season[2]*drawWeight)) - (team2Season[1]*loseWeight)

  team2_total += team2_season_score

  # Incorporate home game advantage into the scorings
  if team1Status == "yes":
    team1_total += (team1_total * homeAdvantage)
  else:
    team2_total += (team2_total * homeAdvantage)

  # Score total of both teams used for calculating win possibilities
  total_score = team1_total + team2_total

  # Calculates win possibilities for both teams
  if total_score > 0:
    team1_win_prob = team1_total / total_score
    team2_win_prob = team2_total / total_score
  else:
    # If no data, assume a draw
    team1_win_prob = 0.5
    team2_win_prob = 0.5

  # Turns into percentage
  team1_win_prob = team1_win_prob * 100
  team2_win_prob = team2_win_prob * 100

  # For cases of 100% win possibility (not possible)
  if team1_win_prob == 100:
    team1_win_prob = 99

  if team2_win_prob == 100:
    team2_win_prob = 99

  # Print final outcomes
  print("\n\n\nCalculating results...\n\n")
  print(f"\nTeam 1 total score: {team1_total}")
  print(f"Team 2 total score: {team2_total}")
  print(f"\n{team1}'s win probability: {team1_win_prob:.2f}%")
  print(f"{team2}'s win probability: {team2_win_prob:.2f}%")

# Get user input for team names
def getTeamInput(teamNames, prompt):
    while True:
        team = input(prompt).strip().lower()  # Convert user input to lowercase

        # Compare lowercase versions of the team names
        if team not in [t.lower() for t in teamNames]:  # Compare lowercase versions of the team names
            print(f"\nInvalid team name. Please pick a valid team from the list.\n")
        else:
            # Return the properly capitalized team name from the original list
            return next(t for t in teamNames if t.lower() == team)

# Get user input for yes/no
def getHomeInput(prompt):
    while True:
        response = input(prompt).strip().lower()
        if response == "yes" or response == "no":
            return response
        else:
            print("\nInvalid input. Please answer with 'yes' or 'no'.\n")

# Main code to fetch and display team data
if response.status_code == 200:

    print("Below contains the list of the 20 Premier League Teams")
    print("You will be asked to choose 2 of the 20 teams for a match prediction\n")

    standings = response.json()
    standingsTotal = standings['standings'][0]["table"]  # Get the standings table
    teamNames = []
    for teamData in standingsTotal:
        teamNames.append(teamData['team']['shortName'])  # Retrieves team names from the standings

    # Split the teamNames list into two parts
    teamNamesFirstHalf = teamNames[:10]
    teamNamesSecondHalf = teamNames[10:20]

    # Print the two arrays of team names
    print(", ".join(teamNamesFirstHalf) + ",")
    print(", ".join(teamNamesSecondHalf))

    print("\n\nPlease pick a team from the above with exact spelling: ")

    # Get user input
    team1 = getTeamInput(teamNames, "Pick Team 1: ")
    team1Status = getHomeInput(f"Is {team1} playing at home? [yes/no]: ")
    team2 = getTeamInput(teamNames, "Pick Team 2: ")
    print("\n\n" + "-"*80)

    # Retrieve the detailed team information for both teams
    team1Info = getTeamInfo(response, team1)
    team2Info = getTeamInfo(response, team2)
    team1Form = team1Info[0]
    team2Form = team2Info[0]

    # Organize team season stats (wins, losses, draws)
    team1Season = (team1Info[1], team1Info[2], team1Info[3])  # (Wins, Losses, Draws)
    team1HomeSeason = (team1Info[4], team1Info[5], team1Info[6]) # Home stats
    team1AwaySeason = (team1Info[7], team1Info[8], team1Info[9]) # Away stats

    team2Season = (team2Info[1], team2Info[2], team2Info[3])  # (Wins, Losses, Draws)
    team2HomeSeason = (team2Info[4], team2Info[5], team2Info[6]) # Home stats
    team2AwaySeason = (team2Info[7], team2Info[8], team2Info[9]) # Away stats

    # Print stats for both teams
    print(f"\n\n{team1}'s current form: " + team1Form)
    print(f"\nSeason stats for {team1}:\nWins: {team1Season[0]}, Losses: {team1Season[1]}, Draws: {team1Season[2]}")
    print(f"\nHome game stats for {team1}:\nWins: {team1HomeSeason[0]}, Losses: {team1HomeSeason[1]}, Draws: {team1HomeSeason[2]}")
    print(f"\nAway game stats for {team1}:\nWins: {team1AwaySeason[0]}, Losses: {team1AwaySeason[1]}, Draws: {team1AwaySeason[2]}")

    print(f"\n\n{team2}'s current form: " + team2Form)
    print(f"\nSeason stats for {team2}:\nWins: {team2Season[0]}, Losses: {team2Season[1]}, Draws: {team2Season[2]}")
    print(f"\nHome game stats for {team2}:\nWins: {team2HomeSeason[0]}, Losses: {team2HomeSeason[1]}, Draws: {team2HomeSeason[2]}")
    print(f"\nAway game stats for {team2}:\nWins: {team2AwaySeason[0]}, Losses: {team2AwaySeason[1]}, Draws: {team2AwaySeason[2]}")

else:
    print("Error:", response.status_code)


# Calls the predictMatch function
prediction = predictMatch(team1Form, team2Form, team1Status, team1Season, team2Season)

Below contains the list of the 20 Premier League Teams
You will be asked to choose 2 of the 20 teams for a match prediction

Liverpool, Arsenal, Nottingham, Chelsea, Man City, Newcastle, Bournemouth, Aston Villa, Fulham, Brighton Hove,
Brentford, Crystal Palace, Man United, Tottenham, West Ham, Everton, Wolverhampton, Leicester City, Ipswich Town, Southampton


Please pick a team from the above with exact spelling: 
Pick Team 1: arsenal
Is Arsenal playing at home? [yes/no]: no
Pick Team 2: liverpool


--------------------------------------------------------------------------------


Arsenal's current form: W,W,D,W,D

Season stats for Arsenal:
Wins: 14, Losses: 2, Draws: 8

Home game stats for Arsenal:
Wins: 8, Losses: 0, Draws: 4

Away game stats for Arsenal:
Wins: 6, Losses: 2, Draws: 4


Liverpool's current form: W,W,W,D,D

Season stats for Liverpool:
Wins: 17, Losses: 1, Draws: 5

Home game stats for Liverpool:
Wins: 8, Losses: 1, Draws: 2

Away game stats for Liverpool:
Wins: 9, Lo